# Text embedding

In [1]:
from tqdm import tqdm
import pandas as pd

f = open('go.obo')
lines = f.readlines()


GO_id = []
GO_name = []
namespace = []
definition = []
is_a = []
part_of = []
for line in tqdm(lines):
    line = line.strip()
    if line.startswith('id:'):
        GO_id.append(line[4:])
    elif line.startswith('name:'):
        GO_name.append(line[5:])
    elif line.startswith('namespace:'):
        namespace.append(line[10:])
    # elif line.startswith('def:'):
    #     definition.append(line[6:])
    # elif line.startswith('is_a:'):
    #     is_a.append(line[5:])
    # elif line.startswith('relationship: part_of'):
    #     part_of.append(line[21:])
    elif line.startswith('is_obsolete: true'):
        GO_id.pop()
        GO_name.pop()
        namespace.pop()
        # definition.pop()
        # is_a.pop()
        # part_of.pop()
        
GO_data = pd.DataFrame([GO_id, GO_name, namespace]).T
# GO_data = pd.DataFrame([GO_id, GO_name, namespace, definition, is_a, part_of]).T
# GO_data.columns = ['GO_id', 'GO_name', 'namespace', 'definition', 'is_a', 'part_of']
GO_data.columns = ['GO_id', 'GO_name', 'namespace']
    
output_file = 'GO_fil.csv'
GO_data.to_csv(output_file, index=False)


100%|██████████| 616743/616743 [00:00<00:00, 2190789.56it/s]


In [1]:
!pip install sentence-transformers
!pip install keras-nlp
!pip install transformers


  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.4.1-cp38-cp38-win_amd64.whl.metadata (27 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl.metadata (58 kB)
  Using cached pillow-10.4.0-cp38-cp38-win_amd64.whl.metadata (9.3 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached regex-2024.9.11-cp38-cp38-win_amd64.whl.metadata (41 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.

In [3]:
!pip install pandas

   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.8 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/10.8 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/10.8 MB 3.4 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/10.8 MB 2.5 MB/s eta 0:00:04
   ------------- -------------------------- 3.7/10.8 MB 3.6 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/10.8 MB 3.6 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/10.8 MB 3.7 MB/s eta 0:00:02
   --------------------- ------------------ 5.8/10.8 MB 3.6 MB/s eta 0:00:02
   ------------------------- -------------- 6.8/10.8 MB 3.7 MB/s eta 0:00:02
   --------------------------- ------------ 7.3/10.8 MB 3.7 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.8 MB 3.6 MB/s eta 0:00:01
   ----------

In [10]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
from torch import nn

model = SentenceTransformer('all-MiniLM-L6-v2')   
df = pd.read_csv('GO_data.csv')

df['text'] = df['definition'].fillna('')

texts = df['text'].tolist()   embeddings = model.encode(texts, convert_to_tensor=True, show_progress_bar=True)   
target_dim = 256

linear_layer = nn.Linear(384, target_dim)

embeddings_300 = linear_layer(embeddings)

embeddings_300 = embeddings_300.detach().cpu().numpy()   
embedding_columns = [f'embedding_{i}' for i in range(embeddings_300.shape[1])]
embeddings_df = pd.DataFrame(embeddings_300, columns=embedding_columns)

df = pd.concat([df, embeddings_df], axis=1)

method_name = 'sentence-bert-linear'
output_file = f'GO_data_output_{method_name}_{target_dim}d.csv'

df.to_csv(output_file, index=False)

print(f"DataFrame saved as '{output_file}' with embeddings of {target_dim} dimensions")


Batches:   0%|          | 0/2112 [00:00<?, ?it/s]

Embeddings saved as 'GO_embeddings_384d.pt' and 'GO_embeddings_384d.npy' with 384 dimensions


# Triples

In [ ]:
from tqdm import tqdm
import pandas as pd

f = open('go.obo')
lines = f.readlines()

nodes1 = []
names1 = []
namespaces = []
relationships = []
nodes2 = []
names2 = []

def append_all_lists():
    nodes1.append(node1)
    names1.append(name1)
    namespaces.append(namespace)
    relationships.append(relationship)
    nodes2.append(node2)
    names2.append(name2)

for line in tqdm(lines):
    line = line.strip()
    if line.startswith('id:'):
                 node1 = line[line.find('GO:'):].strip()
    elif line.startswith('name:'):
                 name1 = line[5:].strip()
    elif line.startswith('namespace:'):
                 namespace = line[10:].strip()
    elif line.startswith('is_a:'):
        relationship = 'is a'
        node2 = line.split('!')[0][5:].strip()
        name2 = line.split('!')[1].strip()
        append_all_lists()
    elif line.startswith('relationship:'):
        relationship = line.split('GO:')[0][13:].strip()
        node2 = line.split('!')[0][line.find('GO:'):].strip()
        name2 = line.split('!')[1].strip()
        append_all_lists()

 GO_GO_data = pd.DataFrame([nodes1, names1, namespaces, relationships, nodes2, names2]).T
GO_GO_data.columns = ['GO_id1', 'GO_name1', 'type', 'relationship', 'GO_id2', 'GO_name2']

output_file = 'GO.csv'
GO_GO_data.to_csv(output_file, index=False)

print(f"DataFrame saved as '{output_file}'")


In [ ]:
 import pandas as pd

 df = pd.read_csv('GO.csv')

 with open('GO_GO_relations.txt', 'w') as f:
    for index, row in df.iterrows():
                 f.write(f"{row['GO_id1']}\t{row['relationship']}\t{row['GO_id2']}\n")

 print('Data saved as GO_GO_relations.txt')


# namespace

In [ ]:
 import pandas as pd

 df = pd.read_csv('GO_GO_data_output.csv')

 namespace_dict = df.set_index('GO_id1')['namespace1'].to_dict()

 df['namespace2'] = df['GO_id2'].map(namespace_dict)

 df.to_csv('GO_GO_data_output.csv', index=False)


# Splitting data

In [ ]:
 import pandas as pd

df = pd.read_csv('GO_GO_relations.txt', sep='\t')

train = df.sample(frac=0.8)
valid = df.drop(train.index).sample(frac=0.5)
test = df.drop(train.index).drop(valid.index)

train.to_csv('train.txt', sep='\t', index=False)
valid.to_csv('valid.txt', sep='\t', index=False)
test.to_csv('test.txt', sep='\t', index=False)

print('Data saved as train.txt, valid.txt, and test.txt')

In [3]:
import pandas as pd

df = pd.read_csv('GO.csv')

 
with open('GO_GO_relations.txt', 'w') as f:
    for index, row in df.iterrows():
        f.write(f"{row['GO_id1']}\t{row['relationship']}\t{row['GO_id2']}\n")
        
print('Data saved as GO_GO_relations.txt')

Data saved as GO_GO_relations.txt


In [ ]:
import pandas as pd


df = pd.read_csv('GO_GO_data_output.csv')

namespace_dict = df.set_index('GO_id1')['namespace1'].to_dict()

df['namespace2'] = df['GO_id2'].map(namespace_dict)

df.to_csv('GO_GO_data_output.csv', index=False)

In [1]:
import pandas as pd

df = pd.read_csv('GO_GO_relations.txt', sep='\t')

train = df.sample(frac=0.8)
valid = df.drop(train.index).sample(frac=0.5)
test = df.drop(train.index).drop(valid.index)

train.to_csv('train.txt', sep='\t', index=False)
valid.to_csv('valid.txt', sep='\t', index=False)
test.to_csv('test.txt', sep='\t', index=False)

print('Data saved as train.txt, valid.txt, and test.txt')




Data saved as train.txt, valid.txt, and test.txt
